In [104]:
import sys
from pathlib import Path
import pandas as pd 
import numpy as np
import keras
import math 
import tensorflow as tf
from keras.layers import LSTM, Dense, Dropout, LeakyReLU, GRU, BatchNormalization, Input, LayerNormalization
from keras.regularizers import l1, l2, l1_l2
from tensorflow.keras.optimizers import SGD
import tensorflow_addons as tfa
from keras.models import Sequential
from keras.callbacks import LearningRateScheduler, EarlyStopping
from keras.preprocessing.sequence import TimeseriesGenerator
sys.path.append(str(Path("../..").resolve()))
from src.constants import model_data_dir, pred_data_dir
from src.utils import use_target

ImportError: cannot import name 'pred_data_dir' from 'src.constants' (/Users/qiushi/workspace/dsi-courses/APlusBernstein-Project/src/constants.py)

In [57]:
df_train = pd.read_csv(model_data_dir / "train_classification.csv")
df_test = pd.read_csv(model_data_dir / "test_classification.csv")
test_dates = pd.to_datetime(df_test["date"])
df_train = df_train.drop(columns=["date"])
df_test = df_test.drop("date", axis=1)
df_train = df_train.loc[:, ~df_train.columns.str.contains("_mv_")]
df_test = df_test.loc[:, ~df_test.columns.str.contains("_mv_")]
df_train = pd.get_dummies(df_train.astype({
    "month": "category",
    "weekday": "category",
    "day": "category",
}), drop_first=True)
df_test = pd.get_dummies(df_test.astype({
    "month": "category",
    "weekday": "category",
    "day": "category",
}), drop_first=True)

## LSTM

In [60]:
win_length = 40
batch_size = 10
num_features = df_train.shape[1] - 3

In [61]:
model = Sequential()
model.add(Input(shape=(win_length, num_features)))
model.add(LSTM(64, return_sequences=True, bias_initializer="zeros", unit_forget_bias=True, kernel_regularizer=l1(1e-4), recurrent_regularizer=l2(2e-4)))
model.add(LeakyReLU(alpha=0.5)) 
model.add(LayerNormalization())
model.add(Dropout(0.3)) 
model.add(LSTM(32, return_sequences=False, bias_initializer="zeros", unit_forget_bias=True, kernel_regularizer=l1(1e-4), recurrent_regularizer=l2(2e-4)))
model.add(LeakyReLU(alpha=0.5)) 
model.add(LayerNormalization())
model.add(Dropout(0.3)) 
model.add(Dense(1, activation="sigmoid", kernel_regularizer=l1_l2(1e-4, 2e-4)))

In [62]:
early_stop = EarlyStopping(monitor = "loss",
                           patience = 5)

step = tf.Variable(0, trainable=False)
schedule = tf.optimizers.schedules.PiecewiseConstantDecay(
    [10000, 15000], [1e-0, 1e-1, 1e-2])
# lr and wd can be a function or a tensor
lr = 1e-1 * schedule(step)
wd = lambda: 1e-4 * schedule(step)
model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=tfa.optimizers.SGDW(
              learning_rate=lr, 
              weight_decay=wd, 
              momentum=0.9),
              metrics=["accuracy"])

## size

In [82]:
y_train_size, x_train_size = use_target(df_train, "sc_1d_fwd_rel_d", "classification")
y_test_size, x_test_size = use_target(df_test, "sc_1d_fwd_rel_d", "classification")

train_generator_size = TimeseriesGenerator(x_train_size, y_train_size, length=win_length, sampling_rate=1, batch_size=batch_size)
test_generator_size = TimeseriesGenerator(x_test_size, y_test_size, length=win_length, sampling_rate=1, batch_size=batch_size)

In [84]:
history = model.fit(train_generator_size, 
                    epochs=20,
                    validation_data=test_generator_size,
                    shuffle=False,
                    callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)])

Epoch 1/20
334/334 [==============================] - 7s 22ms/step - loss: 0.6985 - accuracy: 0.5082 - val_loss: 0.6965 - val_accuracy: 0.4865
Epoch 2/20
334/334 [==============================] - 7s 21ms/step - loss: 0.6989 - accuracy: 0.5100 - val_loss: 0.6965 - val_accuracy: 0.4865
Epoch 3/20
334/334 [==============================] - 7s 20ms/step - loss: 0.6988 - accuracy: 0.5100 - val_loss: 0.6965 - val_accuracy: 0.4865
Epoch 4/20
334/334 [==============================] - 7s 20ms/step - loss: 0.6988 - accuracy: 0.5100 - val_loss: 0.6965 - val_accuracy: 0.4865


In [85]:
size_preds = np.where(model.predict(test_generator_size).flatten() > 0.5, 1, 0)
true = y_test_size[0:len(y_test_size) - win_length ]
np.mean(np.where(size_preds > 0.5, 1, 0) == true)

0.490249187432286

## momentum

In [86]:
y_train_mom, x_train_mom = use_target(df_train, "mom_1d_fwd_rel_d", "classification")
y_test_mom, x_test_mom = use_target(df_test, "mom_1d_fwd_rel_d", "classification")

train_generator_mom = TimeseriesGenerator(x_train_mom, y_train_mom, length=win_length, sampling_rate=1, batch_size=batch_size)
test_generator_mom = TimeseriesGenerator(x_test_mom, y_test_mom, length=win_length, sampling_rate=1, batch_size=batch_size)

In [87]:
model = Sequential()
model.add(Input(shape=(win_length, num_features)))
model.add(LSTM(64, return_sequences=True, bias_initializer="zeros", unit_forget_bias=True, kernel_regularizer=l1(1e-4), recurrent_regularizer=l2(2e-4)))
model.add(LeakyReLU(alpha=0.5)) 
model.add(LayerNormalization())
model.add(Dropout(0.3)) 
model.add(LSTM(32, return_sequences=False, bias_initializer="zeros", unit_forget_bias=True, kernel_regularizer=l1(1e-4), recurrent_regularizer=l2(2e-4)))
model.add(LeakyReLU(alpha=0.5)) 
model.add(LayerNormalization())
model.add(Dropout(0.3)) 
model.add(Dense(1, activation="sigmoid", kernel_regularizer=l1_l2(1e-4, 2e-4)))

In [88]:
early_stop = EarlyStopping(monitor = "loss",
                           patience = 5)

step = tf.Variable(0, trainable=False)
schedule = tf.optimizers.schedules.PiecewiseConstantDecay(
    [10000, 15000], [1e-0, 1e-1, 1e-2])
# lr and wd can be a function or a tensor
lr = 1e-1 * schedule(step)
wd = lambda: 1e-4 * schedule(step)
model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=tfa.optimizers.SGDW(
              learning_rate=lr, 
              weight_decay=wd, 
              momentum=0.9),
              metrics=["accuracy"])

In [90]:
history = model.fit(train_generator_mom, 
                    epochs=20,
                    validation_data=test_generator_mom,
                    shuffle=False,
                    callbacks=[early_stop])

Epoch 1/20
334/334 [==============================] - 7s 20ms/step - loss: 0.7354 - accuracy: 0.5142 - val_loss: 0.7189 - val_accuracy: 0.5417
Epoch 2/20
334/334 [==============================] - 7s 20ms/step - loss: 0.7214 - accuracy: 0.5199 - val_loss: 0.7093 - val_accuracy: 0.5417
Epoch 3/20
334/334 [==============================] - 7s 21ms/step - loss: 0.7156 - accuracy: 0.5250 - val_loss: 0.7042 - val_accuracy: 0.5417
Epoch 4/20
334/334 [==============================] - 7s 20ms/step - loss: 0.7100 - accuracy: 0.5181 - val_loss: 0.7007 - val_accuracy: 0.5417
Epoch 5/20
334/334 [==============================] - 7s 20ms/step - loss: 0.7096 - accuracy: 0.5136 - val_loss: 0.6999 - val_accuracy: 0.5417
Epoch 6/20
334/334 [==============================] - 7s 20ms/step - loss: 0.7049 - accuracy: 0.5262 - val_loss: 0.6970 - val_accuracy: 0.5417
Epoch 7/20
334/334 [==============================] - 7s 19ms/step - loss: 0.7061 - accuracy: 0.5178 - val_loss: 0.7008 - val_accuracy: 0.5417

In [91]:
mom_preds = np.where(model.predict(test_generator_mom).flatten(), 1, 0)
true = y_test_mom[0:len(y_test_mom) - win_length ]
np.mean(mom_preds == true)

0.5449620801733478

## value

In [92]:
y_train_value, x_train_value = use_target(df_train, "value_1d_fwd_rel_d", "classification")
y_test_value, x_test_value = use_target(df_test, "value_1d_fwd_rel_d", "classification")

train_generator_value = TimeseriesGenerator(x_train_value, y_train_value, length=win_length, sampling_rate=1, batch_size=batch_size)
test_generator_value = TimeseriesGenerator(x_test_value, y_test_value, length=win_length, sampling_rate=1, batch_size=batch_size)

In [93]:
model = Sequential()
model.add(Input(shape=(win_length, num_features)))
model.add(LSTM(64, return_sequences=True, bias_initializer="zeros", unit_forget_bias=True, kernel_regularizer=l1(1e-4), recurrent_regularizer=l2(2e-4)))
model.add(LeakyReLU(alpha=0.5)) 
model.add(LayerNormalization())
model.add(Dropout(0.3)) 
model.add(LSTM(32, return_sequences=False, bias_initializer="zeros", unit_forget_bias=True, kernel_regularizer=l1(1e-4), recurrent_regularizer=l2(2e-4)))
model.add(LeakyReLU(alpha=0.5)) 
model.add(LayerNormalization())
model.add(Dropout(0.3)) 
model.add(Dense(1, activation="sigmoid", kernel_regularizer=l1_l2(1e-4, 2e-4)))

In [94]:
early_stop = EarlyStopping(monitor = "loss",
                           patience = 5)

step = tf.Variable(0, trainable=False)
schedule = tf.optimizers.schedules.PiecewiseConstantDecay(
    [10000, 15000], [1e-0, 1e-1, 1e-2])
# lr and wd can be a function or a tensor
lr = 1e-1 * schedule(step)
wd = lambda: 1e-4 * schedule(step)
model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=tfa.optimizers.SGDW(
              learning_rate=lr, 
              weight_decay=wd, 
              momentum=0.9),
              metrics=["accuracy"])

In [95]:
history = model.fit(train_generator_mom, 
                    epochs=20,
                    validation_data=test_generator_mom,
                    shuffle=False,
                    callbacks=[early_stop])

Epoch 1/20
334/334 [==============================] - 15s 27ms/step - loss: 1.0929 - accuracy: 0.5136 - val_loss: 0.9539 - val_accuracy: 0.5423
Epoch 2/20
334/334 [==============================] - 7s 21ms/step - loss: 0.8994 - accuracy: 0.5109 - val_loss: 0.8318 - val_accuracy: 0.5417
Epoch 3/20
334/334 [==============================] - 7s 21ms/step - loss: 0.7996 - accuracy: 0.5109 - val_loss: 0.7619 - val_accuracy: 0.5417
Epoch 4/20
334/334 [==============================] - 6s 19ms/step - loss: 0.7507 - accuracy: 0.5274 - val_loss: 0.7319 - val_accuracy: 0.5417
Epoch 5/20
334/334 [==============================] - 6s 18ms/step - loss: 0.7311 - accuracy: 0.5250 - val_loss: 0.7188 - val_accuracy: 0.5417
Epoch 6/20
334/334 [==============================] - 7s 20ms/step - loss: 0.7206 - accuracy: 0.5271 - val_loss: 0.7101 - val_accuracy: 0.5417
Epoch 7/20
334/334 [==============================] - 6s 18ms/step - loss: 0.7140 - accuracy: 0.5229 - val_loss: 0.7047 - val_accuracy: 0.541

In [99]:
value_preds = np.where(model.predict(test_generator_value).flatten() > 0.5, 1, 0)
true = y_test_value[0:len(y_test_value) - win_length ]
np.mean(value_preds == true)

0.4886240520043337

In [ ]:
pd.DataFrame({
    ""
})

In [103]:
rows = df_test.shape[0]

lstm_preds = pd.DataFrame({
    "sc": df_test["sc_1d_fwd_rel_d"][:rows-win_length], 
    "mom": df_test["mom_1d_fwd_rel_d"][:rows-win_length],
    "value": df_test["value_1d_fwd_rel_d"][:rows-win_length],
    ".pred_sc_lstm": size_preds,
    ".pred_mom_lstm": mom_preds,
    ".pred_value_lstm": value_preds
})

lstm_preds.to_csv(model_data_dir / "lstm_preds.csv", index=False)

,sc,mom,value,.pred_sc_lstm,.pred_mom_lstm,.pred_value_lstm
0,1,0,1,1,1,1
1,0,0,1,1,1,1
2,1,1,1,1,1,1
3,1,1,1,1,1,1
4,0,1,1,1,1,1
...,...,...,...,...,...,...
1841,1,0,0,1,1,1
1842,0,0,0,1,1,1
1843,0,0,0,1,1,1
1844,1,0,1,1,1,1


## GRU 